In [16]:
import pandas as pd
from pathlib import Path

train = Path("../datasets/sdTrain.csv")
test = Path("../datasets/sdTest.csv")

traindf = pd.read_csv(train)
testdf = pd.read_csv(test)


In [8]:
# This is the training dataset and its properties.


print(len(traindf))
traindf.head()

5634


,text,label
0,I have been having migraines and headaches. I ...,308
1,I have asthma and I get wheezing and breathing...,35
2,Signs and symptoms of primary ovarian insuffic...,798
3,"cough,high_fever,breathlessness,family_history...",149
4,"chills,vomiting,high_fever,sweating,headache,n...",596


In [9]:
# This is the testing dataset and its properties.


print(len(testdf))
testdf.head()

1409


,text,label
0,"itching,vomiting,fatigue,weight_loss,high_feve...",541
1,"fatigue,weight_loss,restlessness,sweating,diar...",502
2,"constipation,pain_during_bowel_movements,pain_...",297
3,"chills,vomiting,fatigue,weight_loss,cough,high...",1019
4,"vomiting,yellowish_skin,abdominal_pain,swellin...",33


In [58]:
# This is the dataset we should be using, which allows us to TTS based on our own parameters,
# not the developer's original 0.2 split.

resultdf = pd.concat([traindf , testdf] , ignore_index = True)

print(len(resultdf))
resultdf.head(n=20)

7043


,text,label
0,I have been having migraines and headaches. I can't sleep. My whole body is shaking and shivering. I feel dizzy sometimes.,308
1,"I have asthma and I get wheezing and breathing problems. I also have fevers, headaches, and I feel tired all the time.",35
2,"Signs and symptoms of primary ovarian insufficiency are similar to those of menopause or estrogen deficiency. They include: Irregular or skipped periods, which might be present for years or develop after a pregnancy or after stopping birth control pills Difficulty getting pregnant Hot flashes Night sweats Vaginal dryness Dry eyes Irritability or difficulty concentrating Decreased sexual desire When to see a health care provider If you've missed your period for three months or more, see your health care provider to determine the cause. You can miss your period for a number of reasons — including pregnancy, stress, or a change in diet or exercise habits — but it's best to get evaluated whenever your menstrual cycle changes. Even if you don't mind not having periods, it's advisable to see your provider to find out what's causing the change. Low estrogen levels can lead to bone loss and an increased risk of heart disease.",798
3,"cough,high_fever,breathlessness,family_history,mucoid_sputum",149
4,"chills,vomiting,high_fever,sweating,headache,nausea,diarrhoea,muscle_pain",596
5,"Posterior cortical atrophy symptoms vary among people. Symptoms also can vary over time. They tend to gradually get worse. Common symptoms include having trouble with: Reading, spelling or math. Driving. Getting dressed. Telling the difference between objects that are moving and those that are still. Judging how far away objects are. Using everyday objects or tools. Identifying left from right. Other common symptoms include: Anxiety. Confusion. Changes in behavior and personality. Memory problems may occur later in the disease. Causes The most common cause of posterior cortical atrophy is a form of Alzheimer's disease that's not typical. It affects the back of the brain. Other less common causes include corticobasal degeneration, Lewy body dementia and Creutzfeldt-Jakob disease. Researchers are looking at potential gene variations that may be related to the condition. Risk factors Further study is needed to determine whether the risk factors for Alzheimer's disease may play a role in posterior cortical atrophy. By Mayo Clinic Staff By Mayo Clinic Staff Posterior cortical atrophy care at Mayo Clinic",785
6,"I've been having back pain, a cough, and numbness in my arms and legs. My neck hurts too, and I've been feeling dizzy and off balance.",186
7,"yellowish_skin,nausea,loss_of_appetite,yellowing_of_eyes,family_history",466
8,"joint_pain,neck_pain,knee_pain,hip_joint_pain,swelling_joints,painful_walking",700
9,"An unusual sensation (aura) may precede a temporal lobe seizure, acting as a warning. Not everyone who has temporal lobe seizures has auras, and not everyone who has auras remembers them. The aura is actually the first part of a focal seizure before consciousness is impaired. Examples of auras include: A sudden sense of unprovoked fear or joy A deja vu experience — a feeling that what's happening has happened before A sudden or strange odor or taste A rising sensation in the abdomen, similar to being on a roller coaster Sometimes temporal lobe seizures impair your ability to respond to others. This type of temporal lobe seizure usually lasts 30 seconds to two minutes. Characteristic signs and symptoms include: Loss of awareness of surroundings Staring Lip smacking Repeated swallowing or chewing Unusual finger movements, such as picking motions After a temporal lobe seizure, you may have: A period of confusion and difficulty speaking Inability to recall what occurred during the seizure Unawareness of having had a seizure Extreme sleepiness In extreme cases, what starts as a temporal lobe seizure evolves into a generalized tonic-clonic (grand mal) seizure — featuring convulsions and loss o

In [74]:
# We will now write the resultdf to a csv for development use.

resultdf.to_csv("../datasets/allsymptomsdisease.csv" , index = False)

In [57]:
# Now we're going to look at the data to find inconsistencies and oncongruencies between the
# text and the label number.

resultdf.sort_values(by = 'label',ascending = True)

text  \
717                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt


In [75]:
mydf = pd.read_csv("../datasets/allsymptomsdisease.csv")
mydf = mydf.dropna()
mydf['text'] = mydf['text'].astype(str)
mydf['label'] = pd.to_numeric(mydf['label'], errors='coerce')
mydf = mydf.dropna(subset=['label'])
print(len(mydf))

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(mydf['text']).toarray()
y = mydf['label'].values


7043


In [65]:
# Default split, will be tuned
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [66]:
def build_model(hp):
    model = keras.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(X_train.shape[1],)))
    
    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation='relu'
        ))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1)))
    
    # Output layer for regression
    model.add(layers.Dense(1))
    
    # Optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', tf.keras.metrics.RootMeanSquaredError()]
    )
    return model


In [67]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=20,
    executions_per_trial=1,
    directory='kt_dir',
    project_name='severity_prediction'
)

tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_val, y_val),
             batch_size=32,
             verbose=2)

# Get best model
best_model = tuner.get_best_models(1)[0]


Trial 20 Complete [00h 00m 20s]
val_mae: nan

Best val_mae So Far: 74.64724731445312
Total elapsed time: 00h 07m 16s


c:\Users\conno\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [68]:
loss, mae, rmse = best_model.evaluate(X_val, y_val)
print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20727.3906 - mae: 72.0814 - root_mean_squared_error: 143.5450
MAE: 74.65 | RMSE: 146.62


In [69]:
from sklearn.metrics import r2_score
y_pred = best_model.predict(X_val).flatten()
print("R² Score:", r2_score(y_val, y_pred))


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
R² Score: 0.7530194520950317


In [98]:
def binner(scores):
    output = []
    for score in scores:
        if score > 900:
            output.append(1)
        elif score > 650:
            output.append(2)
        elif score > 400:
            output.append(3)
        elif score > 200:
            output.append(4)
        else:
            output.append(5)
    return output

In [116]:
import numpy as np
mydf2 = pd.read_csv("../datasets/allsymptomsdisease.csv")
mydf2 = mydf2.dropna()
mydf2['text'] = mydf2['text'].astype(str)
mydf2 = mydf2.dropna(subset=['label'])
print(len(mydf2))

mydf2['label'] = binner(mydf2['label'].tolist())

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(mydf2['text']).toarray()
y = mydf2['label'].values - 1



7043


In [118]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [119]:
def build_model2(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation='relu'
        ))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1)))

    model.add(layers.Dense(5, activation='softmax'))

    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [120]:
tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='kt_class_dir',
    project_name='severity_classifier'
)

tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_val, y_val),
             batch_size=32,
             verbose=2)


Trial 8 Complete [00h 00m 29s]
val_accuracy: 0.863733172416687

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 17m 36s

Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_layers
384               |192               |units_0
0.3               |0.2               |dropout_0
rmsprop           |rmsprop           |optimizer
384               |None              |units_1
0.3               |None              |dropout_1
192               |None              |units_2
0.4               |None              |dropout_2

Epoch 1/30
177/177 - 2s - 9ms/step - accuracy: 0.7520 - loss: 0.6535 - val_accuracy: 0.8410 - val_loss: 0.3717
Epoch 2/30
177/177 - 1s - 5ms/step - accuracy: 0.8818 - loss: 0.3010 - val_accuracy: 0.8481 - val_loss: 0.3550
Epoch 3/30
177/177 - 1s - 5ms/step - accuracy: 0.9178 - loss: 0.2211 - val_accuracy: 0.8488 - val_loss: 0.3781
Epoch 4/30
177/177 - 1s - 5ms/step - accuracy: 0.9473 - loss: 0.1589 - val_accura

KeyboardInterrupt: 

In [121]:
best_model = tuner.get_best_models(1)[0]

# Evaluate
loss, accuracy = best_model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy:.4f}")


c:\Users\conno\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2122 - loss: 7.5649 
Validation Accuracy: 0.2165
